<a href="https://colab.research.google.com/github/ozkanyildirim/Dabbel_Algorithms/blob/main/DABBEL_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. EXERCISE:  MATCH PREDICTIONS TO SENSORS**

> ## i. Importing Libraries


In [1]:
# Pandas Operations
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Visualisation Operations
import seaborn as sns
import matplotlib.pyplot as plt

# Generating Random Floating Point Values
from numpy.random import seed
import random
from random import randrange

# System Operations
import sys
import os

pd.options.mode.chained_assignment = None 

> ## ii. Producing Mock Datasets


In [2]:
def df_generator(columns, index):
  """
  Produces Data Frames for the test cases of the algorithm.
  columns : Sensor Data in List() format
  index : index as datetime object
  returns a Pandas Data Frame 
  """ 

  df = pd.DataFrame({'sensor1': pd.Series(columns[0], dtype='float'),
                     'sensor2': pd.Series(columns[1], dtype='float'),
                     'sensor3': pd.Series(columns[2], dtype='float'),
                     'sensor4': pd.Series(columns[3], dtype='float')})
  df = df.set_index(index)
  return df

>> ### a. Generating Data Frames (Maintained with Task)

In [3]:
index_mini_reading = pd.to_datetime(pd.Series(['2020-12-01 00:00:00'\
                                               , '2020-12-01 00:11:34'\
                                               ,'2020-12-01 00:20:00']))
sensor1_mini_read = [20.0, 20.1, 20.0]
sensor2_mini_read = [19.4, 19.4, 19.4]
sensor3_mini_read = [21.3, 21.2, 21.0]
sensor4_mini_read = [20.5, 20.4, 20.6]
columns = [sensor1_mini_read, sensor2_mini_read, sensor3_mini_read, sensor4_mini_read]

df_read = df_generator(columns=columns, index=index_mini_reading)
df_read

,sensor1,sensor2,sensor3,sensor4
2020-12-01 00:00:00,20.0,19.4,21.3,20.5
2020-12-01 00:11:34,20.1,19.4,21.2,20.4
2020-12-01 00:20:00,20.0,19.4,21.0,20.6


In [4]:
index_mini_pred = pd.to_datetime(pd.Series(['2020-12-01 00:00:30', '2020-12-01 00:07:24'\
                                            ,'2020-12-01 00:17:15', '2020-12-01 00:19:00'\
                                            ,'2020-12-01 00:21:30']))
sensor1_mini_pred = [np.nan, 20.2, 19.9, 20.0, 20.1]
sensor2_mini_pred = [19.5, np.nan, 19.6, np.nan, np.nan]
sensor3_mini_pred = [21.1, np.nan, np.nan, np.nan, np.nan]
sensor4_mini_pred = [20.3, 20.2, 20.4, np.nan, 20.5]
columns_pred = [sensor1_mini_pred, sensor2_mini_pred, sensor3_mini_pred, sensor4_mini_pred]

df_pred = df_generator(columns=columns_pred, index=index_mini_pred)
df_pred

,sensor1,sensor2,sensor3,sensor4
2020-12-01 00:00:30,NaN,19.5,21.1,20.3
2020-12-01 00:07:24,20.2,NaN,NaN,20.2
2020-12-01 00:17:15,19.9,19.6,NaN,20.4
2020-12-01 00:19:00,20.0,NaN,NaN,NaN
2020-12-01 00:21:30,20.1,NaN,NaN,20.5


>> ### b. Generating Relatively Large Mock Datasets (For Checking Scaling Effects on Algorith)

In [5]:
def mock_df_generator(row=3000, sensor=500, null_count=500):
  # Sensor1-Sensor500 sensor names
  sensor_list = [f'Sensor{str(i)}' for i in range(1,sensor+1)]

  # Random temprature values from normal distribution with mean 25, variance 3
  temps = np.random.normal(25, 3, size=(row, sensor))

  # Random datetime 
  def random_date(start,l):
    current = start
    while l >= 0:
      current = current + timedelta(days=randrange(30), hours=randrange(10), minutes=randrange(40), seconds=randrange(30))
      yield current
      l-=1

  startDate = datetime(2021, 6, 20,13,00,00)
  index_long = pd.to_datetime([x.strftime("%d/%m/%y %H:%M:%S") for x in reversed(list(random_date(startDate,row-1)))])

  # Wrap up to generate a DataFrame
  df_mock = pd.DataFrame(data=temps, index=index_long, columns=sensor_list, dtype=float).round(decimals=2)

  # Add random Null values
  for col in df_mock.columns:
    null_add = df_mock[col].sample(null_count).index
    df_mock.loc[null_add,col] = np.nan
  
  return df_mock

In [6]:
df_mock_read = mock_df_generator(row=3000, sensor=500, null_count=500)
df_mock_pred = mock_df_generator(row=15000, sensor=500, null_count=2000)

In [7]:
df_mock_read.sample(5)

,Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6,Sensor7,Sensor8,Sensor9,Sensor10,Sensor11,Sensor12,Sensor13,Sensor14,Sensor15,Sensor16,Sensor17,Sensor18,Sensor19,Sensor20,Sensor21,Sensor22,Sensor23,Sensor24,Sensor25,Sensor26,Sensor27,Sensor28,Sensor29,Sensor30,Sensor31,Sensor32,Sensor33,Sensor34,Sensor35,Sensor36,Sensor37,Sensor38,Sensor39,Sensor40,...,Sensor461,Sensor462,Sensor463,Sensor464,Sensor465,Sensor466,Sensor467,Sensor468,Sensor469,Sensor470,Sensor471,Sensor472,Sensor473,Sensor474,Sensor475,Sensor476,Sensor477,Sensor478,Sensor479,Sensor480,Sensor481,Sensor482,Sensor483,Sensor484,Sensor485,Sensor486,Sensor487,Sensor488,Sensor489,Sensor490,Sensor491,Sensor492,Sensor493,Sensor494,Sensor495,Sensor496,Sensor497,Sensor498,Sensor499,Sensor500
1980-08-18 02:30:23,26.81,22.68,16.67,NaN,22.39,26.77,23.54,29.95,28.30,24.61,27.82,NaN,25.22,28.11,24.13,24.34,27.86,22.44,17.09,30.29,NaN,28.85,23.37,28.60,NaN,30.22,NaN,NaN,25.65,NaN,24.65,27.27,23.81,28.65,NaN,25.19,28.82,25.33,25.05,NaN,...,NaN,21.57,23.34,20.90,NaN,26.11,24.56,27.84,NaN,28.91,NaN,NaN,25.87,22.71,24.73,NaN,23.56,27.11,NaN,NaN,28.20,20.70,24.83,20.62,19.82,22.69,23.96,25.95,22.91,29.98,NaN,26.25,NaN,NaN,NaN,21.56,26.19,23.60,29.35,27.52
2021-08-02 04:55:24,23.85,23.44,26.10,21.50,27.72,23.48,22.59,27.66,27.04,28.45,19.93,NaN,22.45,29.50,25.30,21.49,NaN,NaN,27.41,27.53,26.39,20.69,20.42,19.86,20.22,25.20,25.82,22.12,22.24,20.53,NaN,21.61,24.96,21.35,26.13,26.86,28.47,26.79,25.17,28.01,...,21.10,22.14,21.09,22.00,18.44,21.27,NaN,26.16,23.27,25.10,27.31,25.48,24.43,24.29,21.95,24.04,23.39,21.46,23.45,22.88,26.49,23.21,30.36,23.03,NaN,NaN,25.65,24.87,24.97,29.28,NaN,24.46,25.56,NaN,23.78,25.00,26.43,25.47,NaN,NaN
2055-06-13 00:34:02,30.78,25.62,22.55,24.40,27.65,24.58,26.43,21.46,15.09,21.59,28.94,22.82,20.14,26.03,22.76,25.64,23.75,NaN,24.25,25.56,26.47,NaN,27.72,30.67,22.92,25.78,28.88,26.20,23.13,29.37,23.11,23.74,NaN,27.69,23.55,24.22,23.00,25.16,24.49,26.52,...,25.31,21.61,22.17,28.10,27.35,26.89,21.49,28.04,22.47,25.14,24.21,NaN,23.49,NaN,NaN,NaN,26.72,23.90,27.05,28.01,NaN,NaN,24.36,24.63,26.84,19.47,24.48,26.17,NaN,22.02,NaN,27.54,NaN,21.85,NaN,26.65,23.34,26.04,30.27,21.51
2003-10-19 08:19:07,28.89,25.41,NaN,24.25,27.46,26.60,NaN,25.26,26.82,22.82,25.35,24.72,25.50,26.87,28.59,NaN,23.33,25.06,26.88,24.75,29.90,28.02,28.69,22.95,25.29,25.83,26.11,24.59,27.00,28.33,28.94,26.86,26.49,NaN,26.56,26.21,25.11,27.42,22.01,29.20,...,24.42,26.96,23.39,25.29,NaN,27.85,27.81,26.85,21.72,19.48,26.56,24.76,NaN,22.21,NaN,23.01,26.22,NaN,23.44,25.93,24.88,26.44,20.42,22.06,32.34,NaN,NaN,26.69,24.31,22.80,21.62,NaN,25.05,27.34,NaN,22.88,26.09,26.83,23.69,NaN
2063-11-25 15:23:45,26.32,26.48,22.15,NaN,24.49,24.05,23.35,21.05,24.75,27.39,28.30,25.72,25.39,23.94,30.53,26.79,27.15,25.49,20.64,23.61,27.43,19.23,25.16,22.28,26.18,26.18,NaN,19.38,24.24,22.24,22.81,25.90,22.29,26.81,30.05,25.11,23.44,16.71,26.98,23.02,...,26.80,23.81,NaN,NaN,NaN,NaN,28.35,29.00,24.71,27.47,NaN,28.11,25.30,25.31,26.86,28.71,29.53,25.00,27.07,19.28,21.46,26.42,24.82,30.22,21.83,23.77,27.74,24.63,28.44,29.48,30.16,NaN,30.63,21.70,29.20,22.43,27.06,31.42,22.90,28.29


In [8]:
df_mock_pred.sample(5)

,Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6,Sensor7,Sensor8,Sensor9,Sensor10,Sensor11,Sensor12,Sensor13,Sensor14,Sensor15,Sensor16,Sensor17,Sensor18,Sensor19,Sensor20,Sensor21,Sensor22,Sensor23,Sensor24,Sensor25,Sensor26,Sensor27,Sensor28,Sensor29,Sensor30,Sensor31,Sensor32,Sensor33,Sensor34,Sensor35,Sensor36,Sensor37,Sensor38,Sensor39,Sensor40,...,Sensor461,Sensor462,Sensor463,Sensor464,Sensor465,Sensor466,Sensor467,Sensor468,Sensor469,Sensor470,Sensor471,Sensor472,Sensor473,Sensor474,Sensor475,Sensor476,Sensor477,Sensor478,Sensor479,Sensor480,Sensor481,Sensor482,Sensor483,Sensor484,Sensor485,Sensor486,Sensor487,Sensor488,Sensor489,Sensor490,Sensor491,Sensor492,Sensor493,Sensor494,Sensor495,Sensor496,Sensor497,Sensor498,Sensor499,Sensor500
2017-03-01 06:49:29,NaN,31.03,25.99,30.01,25.67,NaN,22.94,25.29,28.33,27.23,27.86,21.24,27.07,28.04,24.73,25.96,21.07,25.20,25.69,21.59,24.16,30.01,24.94,27.88,23.07,NaN,24.53,26.02,22.25,NaN,21.88,24.48,23.66,29.74,24.58,26.02,24.69,25.62,22.55,28.70,...,22.55,23.31,24.92,26.49,27.97,26.17,NaN,27.50,25.82,21.10,NaN,24.77,NaN,NaN,27.12,28.23,26.29,30.69,28.33,27.44,NaN,23.42,NaN,24.33,19.93,26.95,21.15,18.54,25.87,25.05,21.70,NaN,30.54,25.27,NaN,26.05,23.32,24.77,21.64,26.39
2008-10-24 01:07:08,NaN,23.28,25.57,24.79,21.25,20.62,27.87,20.27,22.70,28.97,29.84,NaN,22.09,27.41,24.16,19.71,NaN,21.55,NaN,23.13,NaN,21.51,26.65,32.40,27.30,21.24,NaN,24.50,25.48,26.03,16.66,24.31,22.22,19.74,25.11,24.15,24.06,23.32,29.37,26.97,...,23.24,29.53,29.39,NaN,NaN,NaN,31.11,28.95,25.49,30.84,27.72,27.09,27.60,28.61,NaN,26.45,22.59,23.36,23.35,22.31,21.31,26.41,NaN,27.69,NaN,24.38,19.41,28.88,NaN,26.02,23.61,27.96,29.12,22.77,29.04,22.48,22.97,24.95,27.98,24.66
2060-09-03 10:36:16,21.08,26.30,NaN,NaN,28.91,31.01,24.82,28.03,25.21,24.44,25.77,18.94,21.51,28.23,27.03,22.17,24.13,27.58,24.97,23.89,25.93,NaN,21.50,22.89,23.86,24.16,26.21,29.09,NaN,23.32,25.53,16.15,27.73,NaN,29.76,25.73,30.75,NaN,22.74,27.98,...,26.64,22.35,NaN,22.92,NaN,NaN,NaN,27.80,21.48,21.19,NaN,28.88,28.24,25.90,24.34,23.16,22.71,NaN,22.06,NaN,28.97,24.24,22.17,23.61,NaN,25.06,25.21,25.70,21.74,24.33,29.62,28.50,NaN,25.99,24.20,19.78,20.55,NaN,24.58,27.26
1974-12-22 05:22:48,NaN,29.36,27.72,31.36,28.66,NaN,21.61,NaN,21.18,29.45,22.81,NaN,31.01,26.78,24.35,27.35,25.98,30.95,28.61,NaN,20.17,19.16,21.49,26.72,28.24,20.21,25.02,28.08,NaN,32.25,22.55,26.31,27.23,22.96,19.47,26.11,NaN,29.77,22.85,27.41,...,29.03,27.58,NaN,28.24,23.53,22.58,31.45,24.46,NaN,15.72,24.30,22.07,21.34,22.36,26.57,21.94,27.27,28.45,26.05,25.53,23.96,25.72,29.12,25.94,21.48,28.01,20.83,22.71,27.99,24.76,21.08,NaN,22.98,22.99,NaN,23.58,24.83,28.48,27.19,23.40
2005-11-02 05:26:05,NaN,24.93,28.28,20.78,23.72,NaN,31.11,24.53,22.56,25.15,22.47,18.35,21.77,23.43,22.21,23.65,23.81,26.91,28.30,21.52,29.69,25.77,NaN,26.98,25.19,22.56,NaN,23.85,21.81,25.37,31.05,24.51,30.28,19.67,24.33,29.94,25.13,29.05,NaN,21.03,...,22.60,22.17,28.68,NaN,23.85,26.84,23.36,24.89,NaN,26.02,23.10,26.09,21.60,18.72,NaN,24.50,27.40,20.64,NaN,21.54,23.06,18.05,NaN,26.97,22.27,22.44,30.72,26.62,24.14,24.58,NaN,20.06,30.38,22.03,24.94,27.72,20.99,24.55,24.95,23.53


> ## iii. Designing an Algorithm


>> ### a. Step-1: Concatenate Read and Predict DataFrames

In [9]:
def read_pred_merger(df_read, df_pred):
  """
  Steps will be explained in interview.
  """
  df_read['type'] = 1.0
  df_pred['type'] = 0.0
  df_concat = pd.concat([df_read, df_pred]).sort_index()
  df_concat['time_diff'] = list(pd.Series(df_concat.index).diff())
  df_concat.fillna('nan', inplace=True)
  return df_concat

>> ### b. Step-2: Producing Sensor DataFrames with Check Points

In [10]:
def sensor_df_generator(sensor, df):
  """
  Steps will be explained in interview.
  """
  df_s1 = df[[sensor, 'type', 'time_diff']]
  df_s1 = df_s1.loc[df_s1[sensor]!='nan']
  df_s1['delta_previous'] = df_s1['type'].diff()
  df_s1['delta_forward'] = df_s1['type'].diff(periods=-1)
  df_s1['date_of_action'] = list(df_s1.index)
  df_s1['forward_diff'] = df_s1.time_diff.shift(-1)
  df_s1['forward_diff_2'] = df_s1.time_diff.shift(-2)
  df_s1['rear_diff'] = df_s1.time_diff.shift(1)
  df_s1['rear_diff_2'] = df_s1.time_diff.shift(2)
  df_s1['forward_sensor'] = df_s1[sensor].shift(-1)
  df_s1['rear_sensor'] = df_s1[sensor].shift(1)
  df_s1['forward_date'] = df_s1['date_of_action'].shift(-1)
  df_s1['rear_date'] = df_s1['date_of_action'].shift(1)
  df_s1.fillna('nan', inplace=True)

  return df_s1

>> ### c. Step-3: Applying logic to obtain convenient prediction and read matches

In [11]:
def ultimate_distiller_temp(x):
  """
  Steps will be explained in interview.
  """
  result=[]
  if (x['time_diff']=='nan') & (x['rear_diff']=='nan'):
    result = 'check next'
  elif (x['time_diff']!='nan') & (x['rear_diff']=='nan'):
    result = x['forward_sensor'] if x['time_diff'] > x['forward_diff'] else x['rear_sensor']
  elif x['forward_diff']=='nan':
    result = x['rear_sensor'] if x['time_diff'] > x['rear_diff'] else 'nan'
  elif (x['time_diff']!='nan') & (x['rear_diff']!='nan'):
    if (x['delta_previous']==-1) & (x['delta_forward']==-1) & (x['forward_diff']!='nan'):
      result = x['rear_sensor'] if x['forward_diff'] > x['rear_diff'] else x['forward_sensor']
    elif (x['delta_previous']==0) & (x['delta_forward']==-1):
      result = x['forward_sensor']
    elif (x['delta_previous']==-1) & (x['delta_forward']==0):
      result = x['rear_sensor']  
    elif (x['delta_previous']==1) & (x['delta_forward']==1):
      result = x['rear_sensor'] if x['forward_diff'] > x['rear_diff'] else x['forward_sensor']
    elif (x['delta_previous']==0) & (x['delta_forward']==1):
      result = x['forward_sensor']
    elif (x['delta_previous']==1) & (x['delta_forward']==0):
      result = x['rear_sensor'] 
  return result 

In [12]:
def ultimate_distiller_date(x):
  """
  Steps will be explained in interview.
  """
  result=[]
  if (x['time_diff']=='nan') & (x['rear_diff']=='nan'):
    result = 'check next'
  elif (x['time_diff']!='nan') & (x['rear_diff']=='nan'):
    result = x['forward_date'] if x['time_diff'] > x['forward_diff'] else x['rear_date']
  elif x['forward_diff']=='nan':
    result = x['rear_date'] if x['time_diff'] > x['rear_diff'] else 'nan'
  elif (x['time_diff']!='nan') & (x['rear_diff']!='nan'):
    if (x['delta_previous']==-1) & (x['delta_forward']==-1) & (x['forward_diff']!='nan'):
      result = x['rear_date'] if x['forward_diff'] > x['rear_diff'] else x['forward_date']
    elif (x['delta_previous']==0) & (x['delta_forward']==-1):
      result = x['forward_date']
    elif (x['delta_previous']==-1) & (x['delta_forward']==0):
      result = x['rear_date']  
    elif (x['delta_previous']==1) & (x['delta_forward']==1):
      result = x['rear_date'] if x['forward_diff'] > x['rear_diff'] else x['forward_date']
    elif (x['delta_previous']==0) & (x['delta_forward']==1):
      result = x['forward_date']
    elif (x['delta_previous']==1) & (x['delta_forward']==0):
      result = x['rear_date']
    elif (x['delta_previous']==0) & (x['delta_forward']==0):
      result = 'skipped' 
  return result 

In [13]:
def df_finalizer(df,sensor):
  """
  Steps will be explained in interview.
  """
  df['Date_index'] = df.apply(ultimate_distiller_date, axis=1)
  df['Temperature'] = df.apply(ultimate_distiller_temp, axis=1)
  df = df.loc[df['type']==0.0][['Temperature', sensor,	'Date_index', 'date_of_action']]
  
  #df['diff_act_assg'] = abs(pd.to_datetime(df['Date_index']) - df['date_of_action'])
  #df_finalized = df.groupby(by=['Date_index','Temperature'], as_index=False).min('diff_act_assg')

  df['diff_act_assg'] =abs(pd.to_datetime(df['Date_index']) - df['date_of_action'])
  df_finalized = df[df['diff_act_assg'].isin(df.groupby(by=['Date_index','Temperature']).min()['diff_act_assg'].values)]

  df_finalized['Sensor']=sensor
  df_finalized['Prediction']=df_finalized[sensor]

  return df_finalized

In [14]:
def implement_algorithm(df_reading, df_prediction):
  """
  Steps will be explained in interview.
  """
  sensor_list = df_reading.columns
  df_concat = read_pred_merger(df_reading, df_prediction)
  df_lst = []
  for sens in ['sensor1']:
    df_s = sensor_df_generator(sensor=sens, df=df_concat)
    df_s_finalized = df_finalizer(df_s,sens)
    df_lst.append(df_s_finalized)
  df_ultimate = pd.concat(df_lst, ignore_index=True)
  df_ultimate = df_ultimate.set_index('Date_index')
  df_ultimate = df_ultimate[['Sensor','Temperature','Prediction']]
  return df_ultimate

>> ### d. Step-4: Implementing Algorithm 

In [15]:
def implement_algorithm(df_reading, df_prediction):
  """
  Steps will be explained in interview.
  """
  sensor_list = df_reading.columns
  df_concat = read_pred_merger(df_reading, df_prediction)
  df_lst = []
  for sens in sensor_list:
    df_s = sensor_df_generator(sensor=sens, df=df_concat)
    df_s_finalized = df_finalizer(df_s,sens)
    df_lst.append(df_s_finalized)
  df_ultimate = pd.concat(df_lst, ignore_index=True)
  df_ultimate = df_ultimate.set_index('Date_index')
  df_ultimate = df_ultimate[['Sensor','Temperature','Prediction']]
  return df_ultimate

>>> ### - Implementing Algorithm to Given Datasets

In [16]:
implement_algorithm(df_reading=df_read, df_prediction=df_pred)

,Sensor,Temperature,Prediction
Date_index,,,
2020-12-01 00:11:34,sensor1,20.1,20.2
2020-12-01 00:20:00,sensor1,20,20.0
2020-12-01 00:00:00,sensor2,19.4,19.5
2020-12-01 00:20:00,sensor2,19.4,19.6
2020-12-01 00:00:00,sensor3,21.3,21.1
2020-12-01 00:00:00,sensor4,20.5,20.3
2020-12-01 00:11:34,sensor4,20.4,20.2
2020-12-01 00:20:00,sensor4,20.6,20.5


>>> ### - Implementing Algorithm to Large Datasets

In [ ]:
implement_algorithm(df_reading=df_mock_read, df_prediction=df_mock_pred)

# **2. EXERCISE: WRONG ENCODING**

> ## i. Reading JSON

In [119]:
dabbel = pd.read_json('2000.json')
dabbel.head()

,destination_address,destination_network,device_instance,ip_address,objects
Vendor Proprietary Value,-1,-1,2000,172.17.2.27,"[{'object_instance': 1, 'properties': {'active..."
analog-input,-1,-1,2000,172.17.2.27,"[{'object_instance': 1, 'properties': {'active..."
analog-output,-1,-1,2000,172.17.2.27,"[{'object_instance': 1, 'properties': {'active..."
analog-value,-1,-1,2000,172.17.2.27,"[{'object_instance': 1, 'properties': {'active..."
binary-input,-1,-1,2000,172.17.2.27,"[{'object_instance': 1, 'properties': {'active..."


In [86]:
dabbel.shape

(12, 5)

> ## ii. Designing Text Cleaner

In [75]:
def text_cleaner(df, col):
  """
  Steps will be explained in interview.
  """
  df_mini = df[col].str.strip('\"')
  df_mini = df_mini.str.split('\.([a-zA-Z0-9])').str.join('')
  df_mini = df_mini.str.split('([a-zA-Z0-9])\.').str.join('')
  df_mini = df_mini.str.split('\.(\.)\.').str.join('')
  return df_mini

In [ ]:
test_df = pd.DataFrame(["\".H.E.I.Z.G.R.U.P.P.E\"", "\".T.E.M.P.E.R.A.T.U.R.E...1.F...2.2.5\"",\
                        ".H.E.I.Z.G.R.U.P.P.E", "H.E.I.Z.G.R.U.P.P.E.", \
                        "\"This text is using \"quotes\".\"", "T.h.i.s. is a special test case",\
                        "................", "", "\"\""])

In [123]:
text_cleaner(test_df,0)

0                      HEIZGRUPPE
1              TEMPERATURE.1F.225
2                      HEIZGRUPPE
3                      HEIZGRUPPE
4    This text is using "quotes".
5     This is a special test case
6                          ......
7                                
8                                
Name: 0, dtype: object

> ## iii. Cleaning JSON DataFrame

In [120]:
df_list=[]
cnt=0
for i in range(dabbel.shape[0]):
  for j in range(pd.DataFrame(dabbel.objects[i]).shape[0]):
    df_temp = pd.DataFrame(dabbel.objects[i][j])[['properties']]
    df_list.append(text_cleaner(df=df_temp, col='properties'))
    dabbel.objects[i][j] = dict(df_list[cnt])
    cnt +=1
df_list[0]

active-text                                 NaN
cov-increment                               NaN
description                                    
inactive-text                               NaN
number-of-states                            NaN
object-identifier    Vendor Proprietary Value:1
object-name                          HEIZGRUPPE
object-type                                 NaN
out-of-service                              NaN
present-value                               NaN
priority-array                              NaN
resolution                                  NaN
state-text                                  NaN
status-flags                                NaN
units                                       NaN
Name: properties, dtype: object

> ## iv. Dumping JSON

In [148]:
import json
dabbel_json = dabbel.to_json(orient="index")
dabbel_json[0:500]

'{"Vendor Proprietary Value":{"destination_address":-1,"destination_network":-1,"device_instance":2000,"ip_address":"172.17.2.27","objects":[{"active-text":null,"cov-increment":null,"description":"","inactive-text":null,"number-of-states":null,"object-identifier":"Vendor Proprietary Value:1","object-name":"HEIZGRUPPE","object-type":null,"out-of-service":null,"present-value":null,"priority-array":null,"resolution":null,"state-text":null,"status-flags":null,"units":null},{"active-text":null,"cov-in'

In [149]:
parsed = json.loads(dabbel_json)
json.dumps(parsed, indent=4)[0:500]

'{\n    "Vendor Proprietary Value": {\n        "destination_address": -1,\n        "destination_network": -1,\n        "device_instance": 2000,\n        "ip_address": "172.17.2.27",\n        "objects": [\n            {\n                "active-text": null,\n                "cov-increment": null,\n                "description": "",\n                "inactive-text": null,\n                "number-of-states": null,\n                "object-identifier": "Vendor Proprietary Value:1",\n                "object-name":'